# Demultplexing raw fastq file (post tapioca)

In [42]:
!source activate py36

In [43]:
import sys

sys.path.append('/home/faske/g/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/home/faske/g/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
#import Levenshtein as lv

In [44]:
root = "/home/faske/g/balsam"

In [45]:
cd $root

/working/tfaske/balsam


In [46]:
pwd

'/working/tfaske/balsam'

# Find and manipulate barcode files 
    barcode files need to be in *.csv form
    make lib.txt file to split each individual to own fastq file
    
    RUN THIS BEFOREHAND IN TERMINAL FOR EACH BARCODE FILE
    cut -f 3 -d "," BASA_bc1.csv > lib1_ids.txt

In [49]:
bc_files = []
for bc_dir in ['demult/barcodes/']:
    os.chdir('{}/{}'.format(root, bc_dir))
    files = !find . -name '*.csv'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        bc_files.append(x)
bc_files = sorted(bc_files)

In [50]:
bc_files

['/working/tfaske/balsam/demult/barcodes/BASA_bc1.csv']

In [51]:
#remove Mac line endings 
for bc in bc_files: 
    !tr -d '\r' < $bc > 'bc.out' 
    !mv 'bc.out' $bc

# Find cleaned fastq 
    This finds cleaned fastq files post tapiocaand designates new name assignment for downstream
    once this is changed to fit desired locations, everything should be automatic

In [52]:
fastq_files = []
for contam_dir in ['contam/']:
    os.chdir('{}/{}'.format(root, contam_dir))
    files = !find . -name '*clean.fastq'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        fastq_files.append(x)
fastq_files = sorted(fastq_files)

In [53]:
fastq_files

['/working/tfaske/balsam/contam/BASA1.clean.fastq']

# Run parse
    once above is changed, this should run without change
    This will make shell scripts for each decontam step separately 

In [54]:
demult_dir = os.path.join(root,'demult')

In [55]:
cd $demult_dir

/working/tfaske/balsam/demult


In [56]:
!mkdir 'parse'

In [57]:
parse_perl = '~/g/perl_scripts/parse_barcodes768.pl'

In [58]:
def write_parse_sh(fastq_files,bc_files):
    with open("parse/run_parse.sh", "w") as o:
        o.write("""#!/bin/bash \n""")
        count = 0
        for fastq in fastq_files:
            o.write("""perl {} {} {} D00\n""".format(parse_perl,bc_files[count],fastq))
            count += 1

In [59]:
write_parse_sh(fastq_files,bc_files)

# Run parsh.sh in terminal

    cd /home/faske/g/balsam/demult/parse
    bash run_parse.sh &> parse.out &
    disown -h 
    
    

# PARSE REPORT

In [60]:
report = []
for parse_dir in ['parse/']:
    os.chdir('{}/{}'.format(demult_dir, parse_dir))
    files = !find . -name 'parsereport*'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        report.append(x)
report = sorted(report)

In [61]:
report

['/working/tfaske/balsam/demult/parse/parsereport_BASA1.clean.fastq']

In [62]:
for pr in report:
    print(pr)
    !head -n 4 $pr
    print('\n')

/working/tfaske/balsam/demult/parse/parsereport_BASA1.clean.fastq
Good mids count: 1335683466
Bad mids count: 45498540
Number of seqs with potential MSE adapter in seq: 166411
Seqs that were too short after removing MSE and beyond: 87




# Split parse file to seperate fastq per individual

In [63]:
cd $demult_dir

/working/tfaske/balsam/demult


In [64]:
!mkdir 'fastq'

In [65]:
parse_files = []
for parse_dir in ['parse/']:
    os.chdir('{}/{}'.format(demult_dir, parse_dir))
    files = !find . -name 'parsed*clean.fastq'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        parse_files.append(x)
parse_files = sorted(parse_files)

In [66]:
parse_files

['/working/tfaske/balsam/demult/parse/parsed_BASA1.clean.fastq']

In [67]:
lib_files = []
for bc_dir in ['demult/barcodes/']:
    os.chdir('{}/{}'.format(root, bc_dir))
    files = !find . -name 'lib*.txt'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        lib_files.append(x)
lib_files = sorted(lib_files)

In [68]:
lib_files

['/working/tfaske/balsam/demult/barcodes/lib1_ids.txt']

In [69]:
split_perl = '/home/faske/g/perl_scripts/splitFastq.pl'

In [70]:
cd $demult_dir

/working/tfaske/balsam/demult


In [71]:
def write_split_sh(parse_files,bc_files):
    with open("fastq/run_split.sh", "w") as o:
        o.write("""#!/bin/bash \n""")
        count = 0
        for parse in parse_files:
            o.write("""perl {} {} {}\n""".format(split_perl,lib_files[count],parse))
            count += 1

In [72]:
write_split_sh(parse_files,lib_files)

# Run split.sh in terminal
    cd /home/faske/g/balsam/demult/fastq
    bash run_split.sh &> split.out &
    disown -h
    

# gzip individual fastq files

In [73]:
cd $demult_dir

/working/tfaske/balsam/demult


In [74]:
fq_files = []
for fq_dir in ['fastq']:
    os.chdir('{}/{}'.format(demult_dir, fq_dir))
    files = !find . -name '*fastq'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        fq_files.append(x)
fq_files = sorted(fq_files)

In [75]:
len(fq_files), fq_files[0]

(543, '/working/tfaske/balsam/demult/fastq/BH_QH_1.fastq')

In [76]:
cd $demult_dir

/working/tfaske/balsam/demult


In [77]:
def write_gzip_sh(fq_files):
    with open("fastq/run_gzip.sh", "w") as o:
        o.write("""#!/bin/bash \n""")
        for fq in fq_files:
            o.write("""gzip {} &\n""".format(fq))

In [78]:
write_gzip_sh(fq_files)

# Run gzip.sh in terminal
    cd /home/faske/g/balsam/demult/fastq
    bash run_gzip.sh

# Rename all fastq files 
    dDocent requires *F.fq.gz
    make sure you have this rename:
    conda install -c bioconda rename


In [79]:
!which rename

/home/faske/g/anaconda3/envs/py36/bin/rename


In [80]:
os.chdir('{}/{}'.format(demult_dir, 'fastq'))

In [81]:
pwd

'/working/tfaske/balsam/demult/fastq'

In [82]:
!rename 's/fastq/F.fq/g' *fastq.gz

In [86]:
!ls -l *fastq.gz | wc -l 

ls: cannot access *fastq.gz: No such file or directory
0


In [85]:
!ls -l *F.fq.gz | wc -l 

543


# gzip all raw / intermediate fastq files

In [87]:
cd $root

/working/tfaske/balsam


In [88]:
raw_files = !find . -type f -name '*fastq'
raw_files = [os.path.abspath(x) for x in raw_files]
raw_files = sorted(raw_files)

In [89]:
raw_files

['/working/tfaske/balsam/contam/BASA1.clean.fastq',
 '/working/tfaske/balsam/demult/parse/parsed_BASA1.clean.fastq',
 '/working/tfaske/balsam/demult/parse/parsereport_BASA1.clean.fastq']

In [141]:
def write_gzip_raw_sh(raw_files):
    with open("run_gzip_raw.sh", "w") as o:
        o.write("""#!/bin/bash \n""")
        for f in raw_files:
            o.write("""gzip {} &\n""".format(f))

In [142]:
write_gzip_raw_sh(raw_files)

# run gzip_raw.sh
    cd ~/g/balsam
    bash run_gzip_raw.sh